# Hyperparameter tuning in Python and Model Selection

## Agenda
- Intro to Hyperparameter tuning
- Intro to Hyperopt
- Intro to pySMAC
- Branin function benchmark
- Parameter fitting using Grid search, Random search, Hyperopt and SMAC
- Parameter fitting in xgboost
- Model selection using Hyperopt

### Hyperparameter tuning
Hyperparameter tuning in itself is also an optimization problem. We want to find a configuration of hyperparameters to maximize the model performance. However the relationship between the performance and hyperparameter can not be directly formulated as a set of mathematical equations. It is a black box. Since its a black box and we don't have mathematical equations, we don't have gradients to optimize over. Thereby we cannot use standard optimization methods of gradient decent.

### There are three common methods of hyperparameter tuning: grid search, random search and automatic tuning. 

1. **Grid Search**
    - In grid search, we try a set of configurations of hyperparameters and train the algorithm accordingly, finally choosing the hyperparameter configuration that gives the best performance.
    - In practice, we specify the bounds and steps between values of the hyperparameters, so that it forms a grid of configurations.
    - Typically we start with a limited grid with relatively large steps between parameter values, then extend or make the grid finer at the best configuration and continue searching on the new grid. 

2. **Random Search**
    - In random search, we navigate the grid randomly and it has been shown that one can obtain similar performance to a full grid search. 
    - The authors in #TODO show that if the close-to-optimal region of hyperparameters occupies at least 5% of the search space, then a random search with a certain number of trials (typically 40-60 trials) will be able to find that region with high probability.

3. **Automatic hyperparameter tuning**
    - In both grid or random search, we try configurations randomly. The next trial is independent to all the trials done before. 
    - In contrast, automatic hyperparameter tuning infers knowledge about the relation between the hyperparameter settings and the corresponding model performance in order to make a better choice for the next trial. In other words, it collects the performance at several initial configurations, makes an inference and then decide what configuration to try next.
    - This process is sequential and thereby can not be parallelized.
    - The automatic tuning methods fall into Sequential Model based Optimization - where the black box is modeled in form of an approximate surrogate function. The configuration that maximizes the surrogate function should be tried next. 
    - SMBO methods differ in the way they formulate the surrogate function and then how they optimize the surrogate function.

## Types of Automatic hyperparameter tuning methods
1. **Bayesian Optimization**
    - It uses Gaussian process to model the surrogate and optimizes Expected Improvement which is the expected probability that new trail will improve the current best optimization. 
    - Bayesian Optimization typically gives non-trivial, off-the-grid values for continuous hyperparameters (like the learning rate, regularization coefficient,…) and was shown to beat human performance on some good benchmark datasets. 
    - A well-known implementation of Bayesian Optimization is Spearmint. 
    - However, the process is too slow. Gaussian process is a distribution over functions, so a sample from the distribution is a function. Training a Gaussian Process involves fitting this distribution to the given data, so that it generates functions that are close to the observed data. This is a very slow process.

2. **Sequential Model Algorithmic Configuration (SMAC)**
    - This uses a random forest of regression trees to model the objective function, new points are sampled from the region considered optimal (high Expected Improvement) by the random forest. The python interface is pySMAC. 
    
3. **Tree structured parzen estimator**
    - This is an improved version of SMAC, where two separated models are used to model the posterior. A well-known implementation of TPE is hyperopt.

### Grid search vs Random search
- Grid search is slow and costly. If we have n hyperparameters with each having 2 values, then we have a total of 2^n configurations.
- However, grid search can be easily parallalized, where each core or worker does an independent configuration.
- Random search is simple and effective and is thereby considered de facto method. It can also be parallelized giving results in fewer trials. 
- Grid search can search over the entire search space while Random search can miss some of area.

In [62]:
import numpy as np
import pandas as pd
from time import time
from scipy.stats import randint as sp_randint
from sklearn import preprocessing
from sklearn.metrics import log_loss

from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

import xgboost as xgb

### Intro to hyperopt - (pip install hyperopt, pip install pymongo)

Let's start with a simple example. Say we want to minimize a quadratic funtion.
\begin{align}
\dot{y} & = (x - 1)^2
\end{align}
We chose the parameter space to be (-3, 3)

In [4]:
bst = fmin(fn=lambda x : (x-1)**2, space=hp.uniform('x',-3,3), algo=tpe.suggest, max_evals=1000)
bst

{'x': 0.9990134255047477}

**To break it down the fmin function takes as arguments** 
- The function to minimize. 
- The space over which optimization takes place 
- The algorithm for optimization 
- Number of evaluations the fmin function performs (more evaluations more precise the result)

**Parameter space**
- Uniform
- Categorical choice
- Normal
- qlognormal

### Trials
To see what is happening under the hood, we can use trials object. The trials object allows to store info at each time step.

In [5]:
fspace = {
    'x': hp.uniform('x', -5, 5)
}

def f(params):
    x = params['x']
    val = x**2
    return {'loss': val, 'status': STATUS_OK}

trials = Trials()
best = fmin(fn=f, space=fspace, algo=tpe.suggest, max_evals=50, trials=trials)

print 'best:', best

print 'trials:'
for trial in trials.trials[:2]:
    print trial

best: {'x': -0.06454879705011476}
trials:
{'refresh_time': datetime.datetime(2017, 3, 7, 21, 31, 31, 171000), 'book_time': datetime.datetime(2017, 3, 7, 21, 31, 31, 170000), 'misc': {'tid': 0, 'idxs': {'x': [0]}, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'vals': {'x': [-4.346539596313321]}, 'workdir': None}, 'state': 2, 'tid': 0, 'exp_key': None, 'version': 0, 'result': {'status': 'ok', 'loss': 18.89240646231957}, 'owner': None, 'spec': None}
{'refresh_time': datetime.datetime(2017, 3, 7, 21, 31, 31, 172000), 'book_time': datetime.datetime(2017, 3, 7, 21, 31, 31, 172000), 'misc': {'tid': 1, 'idxs': {'x': [1]}, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'vals': {'x': [-2.172642660363638]}, 'workdir': None}, 'state': 2, 'tid': 1, 'exp_key': None, 'version': 0, 'result': {'status': 'ok', 'loss': 4.720376129631987}, 'owner': None, 'spec': None}


### Intro to pySMAC (pip install pysmac) 

## Branin function

Let's take a simple function to understand the mechanism how each of the method works. We will benchmark each of the method using time and accuracy. 

In [23]:
def branin(params):
    
    x0 = params['x0']
    x1 = params['x1']
    b = (5.1 / (4.*np.pi**2))
    c = (5. / np.pi)
    t = (1. / (8.*np.pi))
    score = 1.*(x1-b*x0**2+c*x0-6.)**2+10.*(1-t)*np.cos(x0)+10
    
    return score

In [24]:
space = {
    'x0' : hp.uniform('x0', -5, 10),
    'x1' : hp.uniform('x1', 0, 15)
}

In [25]:
bst = fmin(branin, space, tpe.suggest, 1000)
bst

{'x0': -3.121364434058067, 'x1': 12.159966735369272}

## Machine learning/ Scikit learn classifier example 

- We will use the iris dataset here for analysis and tune the parameters for Random Forrest classifier. However, I provide free data loader script to use for any dataset 

In [33]:
#Free loader script. We don't use it here
def load_train(traindata, labelCol=None, idCol=None):
    train = pd.read_csv(traindata)
    if labelCol:
        labels = train[labelCol].values
        lbl_enc = preprocessing.LabelEncoder()
        labels = lbl_enc.fit_transform(labels)
        train = train.drop(labelCol, axis=1)
    if idCol:
        train = train.drop(idCol, axis=1)
    
    return train.values, labels.astype('int32')

def load_test(atestdata, idCol=None):
    test = pd.read_csv(testdata)
    if idCol:
        test = test.drop(idCol, axis=1)
    return test.values

def encode_labels(labels):
    lbl_enc = preprocessing.LabelEncoder()
    labels = lbl_enc.fit_transform(labels)
    
    return lbl_enc, lables.astype(int32)

### Load the dataset

In [8]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [9]:
X.shape

(150L, 4L)

In [10]:
y.shape

(150L,)

In [11]:
np.unique(y)

array([0, 1, 2])

### Specify a classifier and a cross validation scheme

In [25]:
#specify a classifier
clf = RandomForestClassifier(n_estimators=20)

#specify a cross-validation scheme
cv = StratifiedKFold(5)

In [34]:
# specify parameters space
param_dist = {"max_depth": [1,2,3,4],
              "bootstrap": [True, False],
              "max_features": [1,2,3,4],
              "min_samples_split": [2,3,4,5],
              "min_samples_leaf": [1,2,3,4],
              "criterion": ["gini", "entropy"]}

# space for hyperopt
space4rf = {
    'max_depth': hp.choice('max_depth', range(1,5)),
    'max_features': hp.choice('max_features', range(1,5)),
    'criterion': hp.choice('criterion', ["gini", "entropy"]),
    'bootstrap' : hp.choice('bootstrap', [True, False]),
    "min_samples_split": hp.choice('min_samples_split', range(2,6)),
    "min_samples_leaf": hp.choice('min_samples_leaf', range(1,5))
}

### Grid search

In [35]:
rfGridCV = GridSearchCV(clf, param_grid=param_dist, cv=cv, n_jobs=-1)
start = time()
rfGridCV.fit(X, y)
print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(rfGridCV.cv_results_['params'])))

GridSearchCV took 252.48 seconds for 1024 candidate parameter settings.


### Random Search

In [36]:
n_iter_search = 20
rfRandomCV = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=cv, n_jobs=-1)

start = time()
rfRandomCV.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))

RandomizedSearchCV took 5.79 seconds for 20 candidates parameter settings.


### Hyperopt 

In [55]:
best = 0.0
def objective(params):
    global best
    clf = RandomForestClassifier(**params)
    acc = cross_val_score(clf, X, y, cv=cv, n_jobs=-1).mean()
    if acc > best:
        best = acc
        print best, params
    return {'loss': -acc, 'status': STATUS_OK}

trials = Trials()
start = time()
hyperoptBest = fmin(objective, space4rf, algo=tpe.suggest, max_evals=20, trials=trials)
print("Hyperopt took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))

0.76 {'max_features': 2, 'criterion': 'entropy', 'min_samples_split': 2, 'bootstrap': True, 'max_depth': 1, 'min_samples_leaf': 1}
0.946666666667 {'max_features': 3, 'criterion': 'entropy', 'min_samples_split': 5, 'bootstrap': True, 'max_depth': 4, 'min_samples_leaf': 3}
0.96 {'max_features': 3, 'criterion': 'gini', 'min_samples_split': 4, 'bootstrap': True, 'max_depth': 3, 'min_samples_leaf': 2}
0.966666666667 {'max_features': 2, 'criterion': 'entropy', 'min_samples_split': 5, 'bootstrap': True, 'max_depth': 4, 'min_samples_leaf': 3}
Hyperopt took 25.34 seconds for 20 candidates parameter settings.


### pySMAC

### Print the best score and the parameters 

In [51]:
rfGridCV.best_score_, rfGridCV.best_params_

(0.96666666666666667,
 {'bootstrap': True,
  'criterion': 'gini',
  'max_depth': 1,
  'max_features': 3,
  'min_samples_leaf': 2,
  'min_samples_split': 2})

In [52]:
rfRandomCV.best_score_, rfRandomCV.best_params_

(0.96666666666666667,
 {'bootstrap': True,
  'criterion': 'entropy',
  'max_depth': 3,
  'max_features': 2,
  'min_samples_leaf': 2,
  'min_samples_split': 5})

In [56]:
hyperoptBest

{'bootstrap': 0,
 'criterion': 1,
 'max_depth': 3,
 'max_features': 1,
 'min_samples_leaf': 2,
 'min_samples_split': 3}

## Xgboost example

In [ ]:
dtrain = xgb.DMatrix(X, y)

def scoreXGB(params):
    print "Training with params : "
    print params
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    
    # watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    model = xgb.train(params, dtrain, num_round)
    predictions = model.predict(dvalid).reshape((X_test.shape[0], 9))
    score = log_loss(y_test, predictions)
    print "\tScore {0}\n\n".format(score)
    return {'loss': score, 'status': STATUS_OK}

spaceXGB = {
             'n_estimators' : hp.quniform('n_estimators', 100, 1000, 1),
             'eta' : hp.quniform('eta', 0.025, 0.5, 0.025),
             'max_depth' : hp.quniform('max_depth', 1, 13, 1),
             'min_child_weight' : hp.quniform('min_child_weight', 1, 6, 1),
             'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
             'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
             'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05),
             'num_class' : 9,
             'eval_metric': 'mlogloss',
             'objective': 'multi:softprob',
             'nthread' : 6,
             'silent' : 1
             }

trials = Trials()
start = time()
hyperoptBest = fmin(scoreXGB, spaceXGB, algo=tpe.suggest, max_evals=20, trials=trials)
print("XGBoost - Hyperopt took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))

## Model selection using hyperopt 

In [63]:
def hyperopt_train_test(params):
    t = params['type']
    del params['type']
    if t == 'naive_bayes':
        clf = BernoulliNB(**params)
    elif t == 'svm':
        clf = SVC(**params)
    elif t == 'dtree':
        clf = DecisionTreeClassifier(**params)
    elif t == 'knn':
        clf = KNeighborsClassifier(**params)
    else:
        return 0
    return cross_val_score(clf, X, y).mean()

space = hp.choice('classifier_type', [
    {
        'type': 'naive_bayes',
        'alpha': hp.uniform('alpha', 0.0, 2.0)
    },
    {
        'type': 'svm',
        'C': hp.uniform('C', 0, 10.0),
        'kernel': hp.choice('kernel', ['linear', 'rbf']),
        'gamma': hp.uniform('gamma', 0, 20.0)
    },
    {
        'type': 'randomforest',
        'max_depth': hp.choice('max_depth', range(1,20)),
        'max_features': hp.choice('max_features', range(1,5)),
        'n_estimators': hp.choice('n_estimators', range(1,20)),
        'criterion': hp.choice('criterion', ["gini", "entropy"]),
        'scale': hp.choice('scale', [0, 1]),
        'normalize': hp.choice('normalize', [0, 1])
    },
    {
        'type': 'knn',
        'n_neighbors': hp.choice('knn_n_neighbors', range(1,50))
    }
])

count = 0
best = 0
def f(params):
    global best, count
    count += 1
    acc = hyperopt_train_test(params.copy())
    if acc > best:
        print 'new best:', acc, 'using', params['type']
        best = acc
    if count % 50 == 0:
        print 'iters:', count, ', acc:', acc, 'using', params
    return {'loss': -acc, 'status': STATUS_OK}

trials = Trials()
best = fmin(f, space, algo=tpe.suggest, max_evals=1500, trials=trials)
print 'best:'
print best

new best: 0.333333333333 using naive_bayes
new best: 0.927287581699 using knn
new best: 0.953839869281 using svm
new best: 0.979983660131 using svm
new best: 0.986928104575 using svm
new best: 0.993464052288 using svm
iters: 50 , acc: 0 using {'normalize': 1, 'n_estimators': 18, 'scale': 0, 'criterion': 'entropy', 'max_features': 4, 'type': 'randomforest', 'max_depth': 13}
iters: 100 , acc: 0.979983660131 using {'kernel': 'linear', 'C': 0.8262032215744757, 'type': 'svm', 'gamma': 12.176832973486928}
iters: 150 , acc: 0.979983660131 using {'kernel': 'linear', 'C': 0.5254848537239812, 'type': 'svm', 'gamma': 14.809575965678553}
iters: 200 , acc: 0.986928104575 using {'kernel': 'linear', 'C': 2.0015050195918827, 'type': 'svm', 'gamma': 10.103045550524602}
iters: 250 , acc: 0.333333333333 using {'alpha': 1.2427011171500757, 'type': 'naive_bayes'}
iters: 300 , acc: 0 using {'normalize': 0, 'n_estimators': 3, 'scale': 0, 'criterion': 'gini', 'max_features': 4, 'type': 'randomforest', 'max_de